<a href="https://colab.research.google.com/github/Hyeonwid/Lyric_DLProject/blob/main/CarlDavidLyricPrj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [43]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("nick-carroll1/lyrics_dataset")
dataset2 = load_dataset("chloeliu/lyrics")
# features are "Artist" "Song" "Lyrics"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Tokenize the text to convert them into numerical inputs that can be fed into the deep learning model. This involves breaking up the text into individual words or subwords, and mapping each token to a unique integer. The tokenization method must be aligned with the chosen model.

In [44]:
Y=dataset2['train']['artist_name']
print(len(Y))
num_classes = len(np.unique(Y))
print(num_classes)

28372
5426


In [45]:
# text = dataset['train']['Lyrics']
text = dataset2['train']['lyrics']
num_words = max([len(t.split(' ')) for t in text])
tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(text)

word_index = tokenizer.word_index
print(word_index)
seq = tokenizer.texts_to_sequences(text)
print(seq[:10])
print(len(max(seq)))
Xmat = tokenizer.sequences_to_matrix(seq, mode="tfidf")
print(Xmat[1][:10])
postpad = pad_sequences(Xmat, padding='post')
print(postpad[5:10])

print(len(postpad) == len(dataset2['train']))

{'<OOV>': 1, 'know': 2, 'like': 3, 'time': 4, 'come': 5, 'go': 6, 'feel': 7, 'yeah': 8, 'away': 9, 'heart': 10, 'life': 11, 'want': 12, 'cause': 13, 'baby': 14, 'right': 15, 'live': 16, 'leave': 17, 'tell': 18, 'night': 19, 'think': 20, 'world': 21, 'need': 22, 'long': 23, 'gonna': 24, 'look': 25, 'break': 26, 'good': 27, 'hold': 28, 'fall': 29, 'hear': 30, 'mind': 31, 'dream': 32, 'say': 33, 'hand': 34, 'wanna': 35, 'home': 36, 'eye': 37, 'little': 38, 'play': 39, 'lose': 40, 'better': 41, 'walk': 42, 'fuck': 43, 'stay': 44, 'head': 45, 'turn': 46, 'believe': 47, 'stand': 48, 'girl': 49, 'change': 50, 'sing': 51, 'things': 52, 'take': 53, 'start': 54, 'face': 55, 'blue': 56, 'tear': 57, 'tonight': 58, 'money': 59, 'inside': 60, 'place': 61, 'song': 62, 'shit': 63, 'bring': 64, 'people': 65, 'love': 66, 'light': 67, 'word': 68, 'stop': 69, 'sweet': 70, 'hard': 71, 'black': 72, 'gotta': 73, 'true': 74, 'remember': 75, 'lonely': 76, 'give': 77, 'kiss': 78, 'thing': 79, 'bitch': 80, 'days

In [46]:
train_split = 0.8
test_split = 0.1

train_ind = int(train_split * len(dataset2['train']))
test_ind = int((1 - test_split) * len(dataset2['train']))

X_train = postpad[:train_ind]
X_valid = postpad[train_ind:test_ind]
X_test = postpad[test_ind:]

Y_ind = range(num_classes)
d = dict(zip(np.unique(Y), Y_ind))
for i in range(len(Y)):
  Y[i] = d[Y[i]]
  
Y_train = Y[:train_ind]
Y_valid = Y[train_ind:test_ind]
Y_test = Y[test_ind:]


In [47]:
X_train.shape # input shape (num_words,)

Y_train = to_categorical(Y_train,num_classes=num_classes)
Y_valid = to_categorical(Y_valid,num_classes=num_classes)
Y_test = to_categorical(Y_test,num_classes=num_classes)

Y_test[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [41]:
# should we standardize (mean/std)
mean = np.mean(X_train, axis=0)

std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_test = (X_test - mean) / std

# RuntimeWarning: invalid value encountered in true_divide

Ignore: stop words nltk testing

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# # from nltk.stem import PorterStemmer
# nltk.download('stopwords')
# nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# example_sent = """This is a sample sentence,
#                   showing off the stop words filtration."""
  
# stop_words = set(stopwords.words('english'))
# # ps = PorterStemmer()
# # use with ps.stem(str)

# word_tokens = word_tokenize(example_sent)
# filtered_sentence = [w for w in word_tokens if w not in stop_words]
# print(word_tokens)
# print(filtered_sentence)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']
['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'filtration', '.']
